# Introduction to sampling-based planning

## Set up notebook

Do imports.

In [1]:
# Rigid body dynamics (pinocchio)
import pinocchio as pin

# Collision checking
import coal

# Visualization (meshcat)
from pinocchio.visualize import MeshcatVisualizer
import meshcat_shapes

# Robot models (robot_descriptions)
from robot_descriptions.loaders.pinocchio import load_robot_description

# Math ("expm" is the matrix exponential function)
import numpy as np
from scipy.linalg import expm, logm
from qpsolvers import solve_qp

# Timing
import time

# Plots
import matplotlib.pyplot as plt

# Interaction
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm

# Other utilities
import itertools

# Suppress the display of very small numbers
np.set_printoptions(suppress=True)

## Load a robot and enable checking for self-collision

Load a robot model.

In [2]:
robot = load_robot_description(
    'iiwa14_description',           # name of robot model
    root_joint=None,                # fixed base
)

Add all possible collision pairs. Since the robot model has no obstacles yet, these pairs are only associated with self-collision.

In [3]:
robot.collision_model.addAllCollisionPairs()

Show all collision pairs.

In [4]:
for i, pair in enumerate(robot.collision_model.collisionPairs):
    print(f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})')

 0 :  0 x  1 (iiwa_link_0_0 x iiwa_link_1_0)
 1 :  0 x  2 (iiwa_link_0_0 x iiwa_link_2_0)
 2 :  0 x  3 (iiwa_link_0_0 x iiwa_link_3_0)
 3 :  0 x  4 (iiwa_link_0_0 x iiwa_link_4_0)
 4 :  0 x  5 (iiwa_link_0_0 x iiwa_link_5_0)
 5 :  0 x  6 (iiwa_link_0_0 x iiwa_link_6_0)
 6 :  0 x  7 (iiwa_link_0_0 x iiwa_link_7_0)
 7 :  1 x  2 (iiwa_link_1_0 x iiwa_link_2_0)
 8 :  1 x  3 (iiwa_link_1_0 x iiwa_link_3_0)
 9 :  1 x  4 (iiwa_link_1_0 x iiwa_link_4_0)
10 :  1 x  5 (iiwa_link_1_0 x iiwa_link_5_0)
11 :  1 x  6 (iiwa_link_1_0 x iiwa_link_6_0)
12 :  1 x  7 (iiwa_link_1_0 x iiwa_link_7_0)
13 :  2 x  3 (iiwa_link_2_0 x iiwa_link_3_0)
14 :  2 x  4 (iiwa_link_2_0 x iiwa_link_4_0)
15 :  2 x  5 (iiwa_link_2_0 x iiwa_link_5_0)
16 :  2 x  6 (iiwa_link_2_0 x iiwa_link_6_0)
17 :  2 x  7 (iiwa_link_2_0 x iiwa_link_7_0)
18 :  3 x  4 (iiwa_link_3_0 x iiwa_link_4_0)
19 :  3 x  5 (iiwa_link_3_0 x iiwa_link_5_0)
20 :  3 x  6 (iiwa_link_3_0 x iiwa_link_6_0)
21 :  3 x  7 (iiwa_link_3_0 x iiwa_link_7_0)
22 :  4 x 

Update robot data.

In [5]:
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Put the robot at its neutral configuration and check for collisions.

In [6]:
# Get the neutral configuration
q = pin.neutral(robot.model)

# Update geometry placements at this configuration
pin.updateGeometryPlacements(
    robot.model,
    robot.data,
    robot.collision_model,
    robot.collision_data,
    q,
)

# Find all collisions
in_collision = pin.computeCollisions(robot.collision_model, robot.collision_data, False)

# Show all pairs in collision
for i, pair in enumerate(robot.collision_model.collisionPairs):
    s = f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})'
    if robot.collision_data.collisionResults[i].isCollision():
        print(f'{s} : COLLISION')
    else:
        print(s)

 0 :  0 x  1 (iiwa_link_0_0 x iiwa_link_1_0) : COLLISION
 1 :  0 x  2 (iiwa_link_0_0 x iiwa_link_2_0)
 2 :  0 x  3 (iiwa_link_0_0 x iiwa_link_3_0)
 3 :  0 x  4 (iiwa_link_0_0 x iiwa_link_4_0)
 4 :  0 x  5 (iiwa_link_0_0 x iiwa_link_5_0)
 5 :  0 x  6 (iiwa_link_0_0 x iiwa_link_6_0)
 6 :  0 x  7 (iiwa_link_0_0 x iiwa_link_7_0)
 7 :  1 x  2 (iiwa_link_1_0 x iiwa_link_2_0) : COLLISION
 8 :  1 x  3 (iiwa_link_1_0 x iiwa_link_3_0)
 9 :  1 x  4 (iiwa_link_1_0 x iiwa_link_4_0)
10 :  1 x  5 (iiwa_link_1_0 x iiwa_link_5_0)
11 :  1 x  6 (iiwa_link_1_0 x iiwa_link_6_0)
12 :  1 x  7 (iiwa_link_1_0 x iiwa_link_7_0)
13 :  2 x  3 (iiwa_link_2_0 x iiwa_link_3_0) : COLLISION
14 :  2 x  4 (iiwa_link_2_0 x iiwa_link_4_0)
15 :  2 x  5 (iiwa_link_2_0 x iiwa_link_5_0)
16 :  2 x  6 (iiwa_link_2_0 x iiwa_link_6_0)
17 :  2 x  7 (iiwa_link_2_0 x iiwa_link_7_0)
18 :  3 x  4 (iiwa_link_3_0 x iiwa_link_4_0) : COLLISION
19 :  3 x  5 (iiwa_link_3_0 x iiwa_link_5_0)
20 :  3 x  6 (iiwa_link_3_0 x iiwa_link_6_0)
21 :  3

Create a hard-coded list of all pairs that are in collision at the neutral configuration (this could be automated). Then, remove these pairs from the list to check.

In [7]:
# Create a list of useless collision pairs (i.e., those pairs that are always
# in collision even at the neutral configuration)
useless_pairs = [0, 7, 13, 18, 22, 25, 26, 27]

# Remove useless collision pairs - it is important that you remove these from
# last to first (i.e., from high to low index) because higher indices will change
# as you remove lower ones
for i in sorted(useless_pairs, reverse=True):
    robot.collision_model.removeCollisionPair(robot.collision_model.collisionPairs[i])

# Update robot data
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Check for collisions again at the neutral configuration to make sure we did this correctly.

In [8]:
# Get the neutral configuration
q = pin.neutral(robot.model)

# Update geometry placements at this configuration
pin.updateGeometryPlacements(
    robot.model,
    robot.data,
    robot.collision_model,
    robot.collision_data,
    q,
)

# Find all collisions
in_collision = pin.computeCollisions(robot.collision_model, robot.collision_data, False)

# Show all pairs in collision
for i, pair in enumerate(robot.collision_model.collisionPairs):
    s = f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})'
    if robot.collision_data.collisionResults[i].isCollision():
        print(f'{s} : COLLISION')
    else:
        print(s)

 0 :  0 x  2 (iiwa_link_0_0 x iiwa_link_2_0)
 1 :  0 x  3 (iiwa_link_0_0 x iiwa_link_3_0)
 2 :  0 x  4 (iiwa_link_0_0 x iiwa_link_4_0)
 3 :  0 x  5 (iiwa_link_0_0 x iiwa_link_5_0)
 4 :  0 x  6 (iiwa_link_0_0 x iiwa_link_6_0)
 5 :  0 x  7 (iiwa_link_0_0 x iiwa_link_7_0)
 6 :  1 x  3 (iiwa_link_1_0 x iiwa_link_3_0)
 7 :  1 x  4 (iiwa_link_1_0 x iiwa_link_4_0)
 8 :  1 x  5 (iiwa_link_1_0 x iiwa_link_5_0)
 9 :  1 x  6 (iiwa_link_1_0 x iiwa_link_6_0)
10 :  1 x  7 (iiwa_link_1_0 x iiwa_link_7_0)
11 :  2 x  4 (iiwa_link_2_0 x iiwa_link_4_0)
12 :  2 x  5 (iiwa_link_2_0 x iiwa_link_5_0)
13 :  2 x  6 (iiwa_link_2_0 x iiwa_link_6_0)
14 :  2 x  7 (iiwa_link_2_0 x iiwa_link_7_0)
15 :  3 x  5 (iiwa_link_3_0 x iiwa_link_5_0)
16 :  3 x  6 (iiwa_link_3_0 x iiwa_link_6_0)
17 :  3 x  7 (iiwa_link_3_0 x iiwa_link_7_0)
18 :  4 x  6 (iiwa_link_4_0 x iiwa_link_6_0)
19 :  4 x  7 (iiwa_link_4_0 x iiwa_link_7_0)


## Add obstacles and enable checking for collision with them

Record how many collision geometries are associated only with the robot.

In [9]:
n_robot_geoms = len(robot.collision_model.geometryObjects)

Define a function that allows us to specify a pose by $x, y, z$ coordinates and by body-fixed yaw ($\psi$), pitch ($\theta$), and roll ($\phi$) angles.

In [10]:
def XYZYPRtoSE3(xyzypr):
    R = pin.utils.rotate('z', xyzypr[3]) @ pin.utils.rotate('y', xyzypr[4]) @ pin.utils.rotate('x', xyzypr[5])
    p = np.array(xyzypr[:3])
    return pin.SE3(R, p)

Add a number of "capsules" as obstacles to the environment. If you want more complicated shapes, it is also possible to load meshes from files.

In [11]:
# Define the pose of each capsule
xyz_ypr = [
    [ 0.4,  0.0, 0.5, 0., 0., np.pi / 2],
    [-0.4, -0.0, 1.0, 0., 0., np.pi / 2],
    [-0.3,  0.0, 0.1, 0., 0., np.pi / 2],
]

# Define the radius and length of each capsule
rad_len = [
    [0.1, 0.4],
    [0.1, 0.4],
    [0.1, 0.4],
]

# Define the color of each capsule (all the same)
rgba = np.array([1.0, 0.2, 0.2, 1.0])

# Add each capsule as an obstacle that is fixed in space
for i, (xyz_ypr_i, rad_len_i) in enumerate(zip(xyz_ypr, rad_len)):
    # Define obstacle
    obs = pin.GeometryObject.CreateCapsule(rad_len_i[0], rad_len_i[1])
    obs.meshColor = rgba
    obs.name = f'obs{i}'
    obs.parentJoint = 0
    obs.placement = XYZYPRtoSE3(xyz_ypr_i)

    # Add obstacle to both the visual model and the collision model
    robot.collision_model.addGeometryObject(obs)
    robot.visual_model.addGeometryObject(obs)

Add collision pairs for all obstacles.

In [12]:
# Iterate over all robot geometries
for i in range(n_robot_geoms):
    # Iterate over all obstacle geometries
    for j in range(n_robot_geoms, len(robot.collision_model.geometryObjects)):
        robot.collision_model.addCollisionPair(pin.CollisionPair(i, j))

# Update robot data
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Show all collision pairs again.

In [13]:
for i, pair in enumerate(robot.collision_model.collisionPairs):
    print(f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})')

 0 :  0 x  2 (iiwa_link_0_0 x iiwa_link_2_0)
 1 :  0 x  3 (iiwa_link_0_0 x iiwa_link_3_0)
 2 :  0 x  4 (iiwa_link_0_0 x iiwa_link_4_0)
 3 :  0 x  5 (iiwa_link_0_0 x iiwa_link_5_0)
 4 :  0 x  6 (iiwa_link_0_0 x iiwa_link_6_0)
 5 :  0 x  7 (iiwa_link_0_0 x iiwa_link_7_0)
 6 :  1 x  3 (iiwa_link_1_0 x iiwa_link_3_0)
 7 :  1 x  4 (iiwa_link_1_0 x iiwa_link_4_0)
 8 :  1 x  5 (iiwa_link_1_0 x iiwa_link_5_0)
 9 :  1 x  6 (iiwa_link_1_0 x iiwa_link_6_0)
10 :  1 x  7 (iiwa_link_1_0 x iiwa_link_7_0)
11 :  2 x  4 (iiwa_link_2_0 x iiwa_link_4_0)
12 :  2 x  5 (iiwa_link_2_0 x iiwa_link_5_0)
13 :  2 x  6 (iiwa_link_2_0 x iiwa_link_6_0)
14 :  2 x  7 (iiwa_link_2_0 x iiwa_link_7_0)
15 :  3 x  5 (iiwa_link_3_0 x iiwa_link_5_0)
16 :  3 x  6 (iiwa_link_3_0 x iiwa_link_6_0)
17 :  3 x  7 (iiwa_link_3_0 x iiwa_link_7_0)
18 :  4 x  6 (iiwa_link_4_0 x iiwa_link_6_0)
19 :  4 x  7 (iiwa_link_4_0 x iiwa_link_7_0)
20 :  0 x  8 (iiwa_link_0_0 x obs0)
21 :  0 x  9 (iiwa_link_0_0 x obs1)
22 :  0 x 10 (iiwa_link_0_0 

## Convert all collision geometries to convex meshes

This isn't strictly necessary. However, I have found that (1) it makes collision checking with coal much faster, and (2) it is useful for the purpose of example when we want to implement our own QP-based collision checker.

Define function to convert each primitive geometry to a convex mesh by sampling its surface.

In [14]:
def primitive_to_convex(shape, n=32):
    """
    Convert a coal primitive shape to a ConvexBase by sampling
    its surface, building a BVHModel from the points, and computing
    the convex hull.
    """
    if isinstance(shape, coal.Box):
        # half-extents
        x, y, z = shape.halfSide
        verts = np.array([
            [ x,  y,  z], [ x,  y, -z], [ x, -y,  z], [ x, -y, -z],
            [-x,  y,  z], [-x,  y, -z], [-x, -y,  z], [-x, -y, -z],
        ])

    elif isinstance(shape, coal.Cylinder):
        r = shape.radius
        h = shape.halfLength
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        circle = np.column_stack([r * np.cos(theta), r * np.sin(theta)])
        top = np.column_stack([circle, np.full(n, h)])
        bot = np.column_stack([circle, np.full(n, -h)])
        verts = np.vstack([top, bot])

    elif isinstance(shape, coal.Cone):
        r = shape.radius
        h = shape.halfLength
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        base = np.column_stack([r * np.cos(theta), r * np.sin(theta),
                                np.full(n, -h)])
        apex = np.array([[0.0, 0.0, h]])
        verts = np.vstack([base, apex])

    elif isinstance(shape, coal.Sphere):
        verts = _sphere_points(shape.radius, n)

    elif isinstance(shape, coal.Ellipsoid):
        rx, ry, rz = shape.radii
        unit = _sphere_points(1.0, n)
        verts = unit * np.array([rx, ry, rz])

    elif isinstance(shape, coal.Capsule):
        r = shape.radius
        h = shape.halfLength
        # hemisphere + cylinder ring
        unit = _sphere_points(1.0, n)
        top_hemi = unit[unit[:, 2] >= 0] * r + [0, 0, h]
        bot_hemi = unit[unit[:, 2] <= 0] * r + [0, 0, -h]
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        ring = np.column_stack([r * np.cos(theta), r * np.sin(theta),
                                np.zeros(n)])
        verts = np.vstack([top_hemi, bot_hemi,
                           ring + [0, 0, h], ring + [0, 0, -h]])

    else:
        raise TypeError(f"Unsupported shape type: {type(shape)}")

    return _points_to_convex(verts)


def _sphere_points(radius, n):
    """Fibonacci sphere sampling."""
    indices = np.arange(n * n // 2)  # enough points
    # Use a simpler grid approach
    u = np.linspace(0, 2 * np.pi, n, endpoint=False)
    v = np.linspace(0, np.pi, n // 2 + 1)
    theta, phi = np.meshgrid(u, v)
    theta, phi = theta.ravel(), phi.ravel()
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    return np.column_stack([x, y, z])


def _points_to_convex(points):
    """Build a BVHModel from a point cloud and compute its convex hull."""
    bvh = coal.BVHModelOBBRSS()
    bvh.beginModel(0, len(points))
    bvh.addVertices(points)
    bvh.endModel()
    bvh.buildConvexHull(True, "Qt")
    return bvh.convex

Apply function to convert all collision geometries to convex meshes.

In [15]:
# Apply function to all geometry objects in collision_model
for geom_obj in robot.collision_model.geometryObjects:
    geom = geom_obj.geometry
    if isinstance(geom, coal.ShapeBase):
        convex = primitive_to_convex(geom)
        geom_obj.geometry = convex
    elif isinstance(geom, coal.BVHModelBase):
        geom.buildConvexHull(True, 'Qt')
        geom_obj.geometry = geom.convex

# Update robot data
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

## Display robot (showing collision geometry instead of visual geometry)

Display robot (and obstacles) in browser.

In [16]:
# Create a visualizer
vis = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
robot.setVisualizer(vis, init=False)
vis.initViewer(open=True)
vis.loadViewerModel()

# Pause to allow meshcat initialization to finish
time.sleep(0.1)

# Choose what to display
vis.displayFrames(False)
vis.displayVisuals(True)
vis.displayCollisions(False)

# Add our own frames to the visualizer because the default frames are hard to see
frames_to_show = [
    'iiwa_link_0',
    'iiwa_link_ee_kuka',
]
for frame in frames_to_show:
    meshcat_shapes.frame(vis.viewer['frames/' + frame], opacity=1.0, axis_length=0.2)

# Pause to allow meshcat initialization to finish
time.sleep(0.1)

# Change the camera view
vis.setCameraPosition(np.array([0., 2., 0.75]))
vis.setCameraTarget(np.array([0., 0., 0.75]))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


Put the robot at its "neutral" configuration.

In [23]:
# Get and show the neutral configuration (most likely all zeros)
q = pin.neutral(robot.model)
print(f'{q = }')

# Do forward kinematics
# - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
pin.forwardKinematics(robot.model, robot.data, q)
# - Compute the placement of all link frames (modifies robot.data but not robot.model)
pin.updateFramePlacements(robot.model, robot.data)

# Show the configuration in the visualizer
vis.display(q)
for frame in frames_to_show:
    frame_id = robot.model.getFrameId(frame)
    vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

q = array([0., 0., 0., 0., 0., 0., 0.])


Create sliders that allow us to change the configuration.

In [18]:
# Callback function that receives all slider values
def on_slider_change(change):
    global q
    q = np.deg2rad(np.array([slider.value for slider in sliders]))
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    vis.display(q)
    for frame in frames_to_show:
        frame_id = robot.model.getFrameId(frame)
        vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

# Create sliders for each joint
sliders = []
for i in range(robot.nq):
    slider = widgets.FloatSlider(
        value=0,
        min=-180,
        max=180,
        step=0.1,
        description=f'Joint {i+1}:',
        continuous_update=True,  # Only update on release
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='600px')  # Set slider width
    )
    # Attach the same callback to each slider
    slider.observe(on_slider_change, names='value')
    sliders.append(slider)

# Display all sliders
display(widgets.VBox(sliders))

# Initialize robot
on_slider_change(None)

Switch from showing "visual" geometry to showing "collision" geometry.

In [19]:
vis.displayVisuals(False)
vis.displayCollisions(True)

## Define functions to check for collision

Define a function that returns true if there is a collision and false otherwise.

In [20]:
def boolean_collision(q):
    pin.updateGeometryPlacements(
        robot.model,
        robot.data,
        robot.collision_model,
        robot.collision_data,
        q)
    return pin.computeCollisions(robot.collision_model, robot.collision_data, False)

Define a function that returns the (minimum) distance to collision in the workspace. (Note that `computeDistances` will also compute the penetration distance in case there is collision - we ignore this result for now.)

In [21]:
def distance_collision(q):
    if boolean_collision(q):
        return 0.
    else:
        # Compute the distance between each collision pair and
        # get the index of the pair with the smallest distance
        i = pin.computeDistances(robot.collision_model, robot.collision_data)
        # Return that smallest distance
        return robot.collision_data.distanceResults[i].min_distance

## Use OMPL to plan a path with RRT-Connect

Prior to using OMPL, you will need to install it (`pip install ompl` in your conda environment – careful, as of right now, the `pip` version does have the python bindings by the `conda-forge` version does not).

Define a function to plan a path with OMPL (this is just an interface to that library).

In [22]:
# Import OMPL-related libraries
from ompl import base as ob
from ompl import geometric as og

# Define function to plan a paths with RRT-Connect
def plan_path_ompl(q_start, q_goal, timeout=5.0, smooth=False, interpolate=False):
    """
    Use OMPL (RRT-Connect) to find a collision-free path from q_start to q_goal.
    Returns a list of configurations (numpy arrays), or None if no path is found.
    """
    
    # Define the state space (one real value per joint)
    space = ob.RealVectorStateSpace(robot.nq)
    
    # Define bounds on each element of the state space (joint limits)
    bounds = ob.RealVectorBounds(robot.nq)
    for i in range(robot.nq):
        bounds.setLow(i, float(robot.model.lowerPositionLimit[i]))
        bounds.setHigh(i, float(robot.model.upperPositionLimit[i]))
    space.setBounds(bounds)
    
    # Define the validity checker
    def is_valid(state):
        # Convert OMPL state to Pinocchio configuration
        q = np.array([state[i] for i in range(robot.nq)])
        # Check collision
        return not boolean_collision(q)

    # Set up the planning problem
    ss = og.SimpleSetup(space)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn(is_valid))

    # Choose the resolution at which edges will be checked for collision
    # (the default is 0.01). Note that this is a PERCENT of "the space's
    # maximum extent" and is not an absolute distance.
    # 
    # For a (bounded) RealVectorStateSpace, the "maximum extent" is the
    # distance between the two most distant corners of the bounding box.
    ss.getSpaceInformation().setStateValidityCheckingResolution(0.01)

    # Set start and goal
    start = ob.State(space)
    goal = ob.State(space)
    for i in range(robot.nq):
        start()[i] = float(q_start[i])
        goal()[i] = float(q_goal[i])
    ss.setStartAndGoalStates(start, goal)

    # Choose planner (in this case, RRT-Connect)
    planner = og.RRTConnect(ss.getSpaceInformation())
    ss.setPlanner(planner)

    # Solve
    solved = ss.solve(timeout)
    if not solved:
        print('No solution found.')
        return None

    # Apply path smoothing, if desired
    if smooth:
        ss.simplifySolution()
    
    # Extract path
    path = ss.getSolutionPath()

    # Apply interpolation to add intermediate waypoints, if desired, at
    # the same resolution as the edge collision checks
    if interpolate:
        path.interpolate()

    # Convert list of waypoints to numpy array
    return [np.array([path.getState(i)[j] for j in range(robot.nq)]) for i in range(path.getStateCount())]

ModuleNotFoundError: No module named 'ompl'

Apply function to plan a path. Then, visualize the path in the display.

In [ ]:
# Define start and goal configurations
q_start = np.zeros(robot.nq)
q_goal = np.array([0., -1.5, 0., -1.0, 0., 0., 0.])

# Plan a path
path = plan_path_ompl(q_start, q_goal, smooth=True, interpolate=True)

# Visualize the result (checking for collision to be sure)
if path is not None:
    for q in path:
        vis.display(q)
        if boolean_collision(q):
            raise Exception('collision!')
        time.sleep(0.025)

Debug:   RRTConnect: Planner range detected to be 2.797105
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (3 start + 2 goal)
Info:    Solution found in 0.002404 seconds
Info:    SimpleSetup: Path simplification took 0.010667 seconds and changed from 4 to 5 states


## Use our own implementation of RRT-Connect to plan a path

Define a simple tree data structure.

Here is how to create a root node with some configuration `q_start`:
```python
tree = Node(q_start)
```

Here is how to create a child of this root node with some other configuration `q1`:
```python
n1 = Node(q1, parent=tree)
```

Here is how to iterate through the entire tree:
```python
print('ALL NODES IN TREE')
for n in tree:
    print(f' q = {q}')
```

In [ ]:
class Node:
    def __init__(self, q, parent=None):
        self.q = q
        self.parent = parent
        self.children = []
        if parent is not None:
            parent.children.append(self)
    
    def __iter__(self):
        yield self
        for child in self.children:
            yield from child

Implement RRT-Connect.

In [ ]:
def find_closest_node_in_tree(tree, q):
    """
    Returns the node in the tree whose configuration is closest to q.
    """
    
    # FIXME
    return tree

def boolean_collision_edge(q1, q2, res=1e-3):
    """
    Returns True if the straight-line path from q1 to q2 has a collision
    (checked at resolution 1e-3) and False otherwise.
    """
    
    # FIXME
    return False

def plan_path(q_start, q_goal, max_iters=1000):
    """
    Returns a path - a 2d numpy array with a waypoint in each row - from
    q_start to q_goal, or None if no path is found before the maximum number
    of iterations has been reached.
    """

    # Create trees rooted at start and goal
    # FIXME

    # Iterate for at most max_iters iterations
    for iter in range(max_iters):
        pass # <-- FIXME (remove this when you add your own implementation)

        # Sample a random configuration that satisfies joint limits, keeping
        # it only if not in collision
        # FIXME
        
        # Try to connect the configuration with a collision-free edge to the
        # closest node in the forward tree
        # FIXME
        
        # Try to connect the configuration with a collision-free edge to the
        # closest node in the backward tree
        # FIXME
        
        # If the configuration was connected to both forward and backward trees,
        # then a solution has been found - return it as a list of waypoints
        # FIXME
    
    # Declare failure if no path was found
    print('FAILURE (after {max_iters} iterations)')
    return None

Apply implementation of RRT-Connect to plan a path.

In [ ]:
# Define start and goal configurations
q_start = np.array([0.0, 0.0])
q_goal = np.array([-1.5, -1.0])

# Plan path
path = plan_path(q_start, q_goal, max_iters=1000)

Define a function to interpolate the path.

In [ ]:
def interpolate_path(path, res=1e-2):
    """
    Adds intermediate waypoints so that the distance between each consecutive
    waypoint is at most res. Should return a *copy* of the path with the result
    - should not modify the path in place.
    """
    
    # FIXME
    return path

Define a function to smooth the path.

In [ ]:
def get_path_lengths(path):
    """
    Returns a 1d array with the same length as path and with the
    cumulative sum of path lengths in each entry.

    Suppose path has waypoints q1, ..., qn. Then, the array that
    is returned should be as follows:

    [
        0.,
        distance from q1 to q2,
        (distance from q1 to q2) + (distance from q2 to q3),
        ...
    ]
    """

    # FIXME
    return np.zeros(len(path))

def smooth_path(path, max_iters=100):
    """
    Applies shortcutting max_iters times to smooth the path. Should
    return a *copy* of the path with the result - should not modify
    the path in place.
    """
    # Compute path lengths
    d = get_path_lengths(path)

    # Iterate max_iters times
    for i in range(max_iters):
        # Sample two points q1 and q2 along the path, keeping them only
        # if they are not on the same edge
        # FIXME
        
        # Continue only if the straight-line path between q1 and q2 is
        # collision-free
        # FIXME

        # Replace all waypoints between q1 and q2 with the new straight-
        # line path
        # FIXME
        
        # Recompute path lengths
        d = get_path_lengths(path)
    
    return path

Visualize the original path after interpolation.

In [ ]:
# Interpolate original path
path_original = interpolate_path(path)

# Visualize the result (checking for collision to be sure)
for q in path_original:
    vis.display(q)
    if boolean_collision(q):
        raise Exception('collision!')
    time.sleep(0.025)

Visualize the smoothed path after interpolation.

In [ ]:
# Smooth original path
path_smoothed = smooth_path(path)

# Interpolate smoothed path
path_smoothed = interpolate_path(path_smoothed)

# Visualize the result (checking for collision to be sure)
for q in path_smoothed:
    vis.display(q)
    if boolean_collision(q):
        raise Exception('collision!')
    time.sleep(0.025)